# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Eitner  ->  P. Eitner  |  ['P. Eitner']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
Arxiv has 58 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.10597


extracting tarball to tmp_2403.10597... done.


Found 87 bibliographic references in tmp_2403.10597/46859corr.bbl.
Retrieving document from  https://arxiv.org/e-print/2403.11635


extracting tarball to tmp_2403.11635... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.10597-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.10597) | **Detailed chemical composition of the globular cluster Sextans A GC-1 on  the outskirts of the Local Group**  |
|| A. Gvozdenko, et al. -- incl., <mark>P. Eitner</mark> |
|*Appeared on*| *2024-03-19*|
|*Comments*| *14 pages, 8 figures*|
|**Abstract**| The chemical composition of globular clusters (GCs) across the Local Group provides information on chemical abundance trends. The host galaxy, Sextans A, is a low-surface-brightness dwarf irregular galaxy located on the edge of the Local Group.We derive the dynamical mass of the GC together with the mass-to-light ratio and the abundances of the ${\alpha}$ (Mg, Ca, Ti, Si), Fe-peak (Fe, Cr, Mn, Sc, Ni), and heavy elements (Ba, Cu, Zn, Eu). Abundance ratios were determined from the analysis of an optical integrated-light spectrum of Sextans A GC-1, obtained with UVES on the VLT. We apply non-local thermodynamic equilibrium (NLTE) corrections to Mg, Ca, Ti, Fe, and Ni. The GC appears to be younger and more metal-poor than the majority of the GCs of the Milky Way, with an age of 8.6$\pm$2.7 Gyr and $\text{[Fe/H]}=-2.14\pm0.04$ dex. The calculated dynamical mass is $(5.18 \pm1.62) \times 10^5 M_{\odot}$, which results in an atypically high value of the mass-to-light ratio, 4.35$\pm$1.40 M$_{\odot}$/L$_{V \odot}$. Sextans A GC-1 has varying $\alpha$ elements - the Mg abundance is extremely low ($\text{[Mg/Fe]}=-0.79\pm0.29$), Ca and Ti are solar-scaled or mildly enhanced, and Si is enhanced. This makes the mean $\alpha$ abundance (excluding Mg) to be enhanced. The Fe-peak elements are consistent with scaled-solar or slightly enhanced abundances. Ba and Cu have sub-solar abundance ratios, while Zn and Eu are consistent with their upper limits being solar-scaled and enhanced. The composition of Sextans A GC-1 resembles the overall pattern and behaviour of GCs in the Local Group. The anomalous values are the mass-to-light ratio and the depleted abundance of Mg. There is no definite explanation for such an extreme abundance value. Variations in the initial mass function or the presence of an intermediate-mass black hole might explain the high mass-to-light ratio value. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.11635-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.11635) | **The FAST Galactic Plane Pulsar Snapshot Survey: V. PSR J1901+0658 in a  double neutron star system**  |
|| W. Q. Su, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-03-19*|
|*Comments*| *6 pages, 6 figures, submitted to MNRAS*|
|**Abstract**| Double neutron star (DNS) systems offer excellent opportunities to test gravity theories. We report the timing results of a pulsar, PSR J1901+0658, discovered in the FAST Galactic Plane Pulsar Snapshot (GPPS) survey. Based on timing observations by FAST over 5 years, we obtain the phase-coherent timing solutions and derive the precise measurements of its position, spin parameters, orbital parameters, and also the dispersion measure (DM). It has a period of 75.7 ms, a period derivative of 2.169(6)$\times 10^{-19}$ s s$^{-1}$, and the characteristic age of 5.5 Gyr. This pulsar is in an orbit with a period of 14.45 days and an eccentricity of 0.366. One post-Keplerian parameter, periastron advance, has been well measured as being 0.00531(9) deg/yr, from which the total mass of this system is derived to be 2.79(7) $M_{\sun}$. The pulsar has the upper limit of mass of 1.68 $M_{\sun}$, so the lower limit for the companion mass is 1.11 $M_{\sun}$. Because PSR J1901+0658 is a partially recycled pulsar in an eccentric binary orbit with such a large companion mass, it should be in a DNS system according to the evolution history of the binary system. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.10597.md
    + _build/html/tmp_2403.10597/./other_elem_GC.png
    + _build/html/tmp_2403.10597/./alpha_elem_GC.png
    + _build/html/tmp_2403.10597/./Mg45.png
    + _build/html/tmp_2403.10597/./Mg47.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Detailed chemical composition of the globular cluster Sextans A GC-1 on the outskirts of the Local Group

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.10597-b31b1b.svg)](https://arxiv.org/abs/2403.10597)<mark>Appeared on: 2024-03-19</mark> -  _14 pages, 8 figures_

</div>
<div id="authors">

A. Gvozdenko, et al. -- incl., <mark>P. Eitner</mark>

</div>
<div id="abstract">

**Abstract:** The chemical composition of globular clusters (GCs) across the Local Group provides information on chemical abundance trends. Studying GCs in isolated systems in particular provides us with important initial conditions plausibly unperturbed by mergers and tidal forces from the large Local Group spirals. We present a detailed chemical abundance analysis of Sextans A GC-1. The host galaxy, Sextans A, is a low-surface-brightness dwarf irregular galaxy located on the edge of the Local Group. We derive the dynamical mass of the GC together with the mass-to-light ratio and the abundances of the $\alpha$ , Fe-peak, and heavy elements. Abundance ratios were determined from the analysis of an optical integrated-light spectrum of Sextans A GC-1, obtained with UVES on the VLT. We apply non-local thermodynamic equilibrium (NLTE) corrections to Mg, Ca, Ti, Fe, and Ni. The GC appears to be younger and more metal-poor than the majority of the GCs of the Milky Way, with an age of 8.6 $\pm$ 2.7 Gyr and $\text{[Fe/H]}=-2.14\pm0.04$ dex. The calculated dynamical mass is $M_{dyn}=(5.18 \pm1.62) \times 10^5 M_{\odot}$ , which results in an atypically high value of the mass-to-light ratio, 4.35 $\pm$ 1.40 M $_{\odot}$ /L $_{V \odot}$ . Sextans A GC-1 has varying $\alpha$ elements -- the Mg abundance is extremely low, Ca and Ti are solar-scaled or mildly enhanced, and Si is enhanced. The measured values are $\text{[Mg/Fe]}=-0.79\pm0.29$ , $\text{[Ca/Fe]}=+0.13\pm0.07$ , $\text{[Ti/Fe]}=+0.27\pm0.11$ , and $\text{[Si/Fe]}=+0.62\pm0.26$ , which makes the mean $\alpha$ abundance (excluding Mg) to be enhanced $\text{[<Si,Ca,Ti>/Fe]}_{\text{NLTE}}=+0.34\pm0.15$ . The Fe-peak elements are consistent with scaled-solar or slightly enhanced abundances: $\text{[Cr/Fe]}=+0.31\pm0.18$ , $\text{[Mn/Fe]}=+0.19\pm0.32$ , $\text{[Sc/Fe]}=+0.22\pm0.22$ , and $\text{[Ni/Fe]}=+0.02\pm0.12$ . The heavy elements measured are Ba, Cu, Zn, and Eu. Ba and Cu have sub-solar abundance ratios ( $\text{[Ba/Fe]}=-0.48\pm0.21$ and $\text{[Cu/Fe]}<-0.343$ ), while Zn and Eu are consistent with their upper limits being solar-scaled and enhanced, $\text{[Zn/Fe]}<+0.171$ and $\text{[Eu/Fe]}<+0.766$ . The composition of Sextans A GC-1 resembles the overall pattern and behaviour of GCs in the Local Group. The anomalous values are the mass-to-light ratio and the depleted abundance of Mg. There is no definite explanation for such an extreme abundance value. Variations in the initial mass function or the presence of an intermediate-mass black hole might explain the high mass-to-light ratio value.

</div>

<div id="div_fig1">

<img src="tmp_2403.10597/./other_elem_GC.png" alt="Fig7" width="100%"/>

**Figure 7. -** Fe-peak and heavy elements plotted against iron abundance, [Fe/H]. Top: [Sc/Fe] vs [Fe/H]. Second row: [Cr/Fe] vs [Fe/H]. Third row: [Mn/Fe] vs [Fe/H]. Fourth row: [Ba/Fe] vs [Fe/H]. Bottom: [Ni/Fe] vs [Fe/H].  Symbols are the same as in Figure \ref{Fig_alpha_elemVSfe}.
 (*Fig_other_elemVSfe*)

</div>
<div id="div_fig2">

<img src="tmp_2403.10597/./alpha_elem_GC.png" alt="Fig6" width="100%"/>

**Figure 6. -** Individual $\alpha$ elements plotted against iron abundance, [Fe/H]. Top: [Mg/Fe] vs [Fe/H]. Second row: [Ca/Fe] vs [Fe/H]. Third row: [Ti/Fe] vs [Fe/H]. Fourth row: [Si/Fe] vs [Fe/H]. Bottom: $\text{[<Si,Ca,Ti>/Fe]}$ vs [Fe/H]. Turquoise rhombus refers to this work. Purple circles show the NLTE abundance of the GCs in the Local Group from [Larsen, Eitner and Magg (2022)]();
some of these Local Group GCs are highlighted in red symbols and introduced in the legends accordingly, e.g. N147 PA-1 is a red circle, Fornax 3 is a red star, M31 358-219 is a red square,
and red crosses and black Xs present MW disc abundances from [Reddy, Tomkin and Lambert (2003)](), [Reddy and Lambert (2006)](), respectively.
Black open circles are the MW disc abundances from [Bensby, et. al (2005)]().
Blue squares and stars belong to LMC bar and inner disc abundances presented by  ([Hill, Primas and Cole 2013]()) ).
 (*Fig_alpha_elemVSfe*)

</div>
<div id="div_fig3">

<img src="tmp_2403.10597/./Mg45.png" alt="Fig1.1" width="50%"/><img src="tmp_2403.10597/./Mg47.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Variation in [Mg/Fe] test. Top: Spectral window of the Mg I line at 4571 Å. Bottom: Spectral window of the Mg I line at 4703 Å.
The best-fit model is in blue, the solar value of [Mg/Fe] is in red, the enhanced [Mg/Fe] abundance is in green, and the observed spectrum is in grey. (*Fig_Mg_test*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.10597"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

391  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
